In [64]:
import os
import pandas as pd
import pickle
from tqdm.auto import tqdm
import json
import glob

In [65]:
class IP_debug():
    def __init__(self, ip):
        global train_dataset
        global test_dataset
        global train_multi
        global test_multi
        global train_data
        global test_data
        global train_raw
        global test_raw
        global train_key
        global test_key
        global min_data
        
        self.type = 'train' if 'train' in ip else 'test' if 'test' in ip else 'none'
        self.ip = ip.split('_')[1] if self.type == 'train' else ip.split('_')[0]
        self.idx_quan = dict()
        print(self.ip)
        self.file_name = '_'.join(ip.split('_')[2:]) if self.type == 'train' else '_'.join(ip.split('_')[1:])
        check = train_dataset if self.type == 'train' else test_dataset if self.type == 'test' else dict()

        tmp_df = check[self.file_name]
        tmp_df['source'] = tmp_df['source'].str.replace(r'*', '')
        tmp_df['destination'] = tmp_df['destination'].str.replace(r'*', '')
        self.flow_data = tmp_df[(tmp_df['source'] == self.ip) | (tmp_df['destination'] == self.ip)]

        self.flow_data.fillna(-1, inplace=True)
        
        if self.type == 'train':
            self.multi_set = set(train_multi[ip]) if type(train_multi[ip]) == list else train_multi[ip]
        elif self.type == 'test':
            self.multi_set = set(test_multi[ip]) if type(test_multi[ip]) == list else test_multi[ip]
        else:
            self.multi_set = set()
        
        self.quan_index = dict()
        self.quan_label = dict()
        self.idx_index = dict()
        
        type_data = train_data if self.type == 'train' else test_data if self.type == 'test' else []
        type_key = train_key if self.type == 'train' else test_key if self.type == 'test'else []
        
        now = 0
        
        for idx, key in enumerate(type_key):
            label, tmp_key, file = key.split('+')
            
            if self.ip == tmp_key and self.file_name == file.split('.')[0]:
                quan = type_data[idx]
                
                if quan not in self.quan_index:
                    self.quan_index[quan] = []
                    self.quan_label[quan] = []
                
                self.idx_quan[now] = quan
                self.quan_index[quan].append(now)
                self.quan_label[quan].append(label)
                self.idx_index[now] = idx
                now += 1
        
        self.total_len = now
    
    def get_idx_quan(self, idx):
        return self.idx_quan[idx]
    
    def get_multi_set(self):
        return self.multi_set

    def get_quan_index(self, quan):
        return self.quan_index[quan]

    def get_quan_label(self, quan):
        return self.quan_label[quan]
    
    def find_profile(self, quan):
        idea_bank = set()
        
        index_list = self.quan_index[quan]
        type_raw = train_raw if self.type == 'train' else test_raw if self.type == 'test' else []
        
        for idx in index_list:
            idea_bank.add(type_raw[self.idx_index[idx]])
        
        return idea_bank
    
    def find_profile_idx(self, idx):
        type_raw = train_raw if self.type == 'train' else test_raw if self.type == 'test' else []
        tmp = type_raw[self.idx_index[idx]]
        result = []
        for i in tmp[:13]:
            result.append(round(i, 5))
        return result
    
    def get_flow_idx(self, idx):
        global min_data
        selected_data = self.flow_data.iloc[idx:idx+min_data]
        
        print(idx, len(self.flow_data))
        
        total_data = []
        check = False
        if 'first' in self.flow_data.columns:
            check = True
        for _, data in selected_data.iterrows():
            for idx, d in enumerate(data):
                if check:
                    if idx == 5 or idx == 6:
                        continue
                total_data.append(d)

        return total_data
    
    def generate_html(self):
        # 미리 모든 find_profile_idx 결과를 계산
        profile_data = {idx: self.find_profile_idx(idx) for idx in self.idx_index.keys()}
        
        flow_data = {idx: self.get_flow_idx(idx) for idx in self.idx_index.keys()}
        ip_ = self.ip
        
        html_content = '<html><head>'
        html_content += '<script>'
        html_content += 'var profileData = ' + str(profile_data).replace("'", '"') + ';\n'
        html_content += 'var flow_data = ' + str(flow_data).replace("'", '"') + ';\n'
        html_content += 'var ip_ = ' + json.dumps(ip_) + ';\n'
        html_content += ''' 
        
        function toggleData(quan) {
            var dataDiv = document.getElementById('data-' + quan);
            console.log(dataDiv)
            if (dataDiv) {
                dataDiv.style.display = dataDiv.style.display === 'none' ? 'block' : 'none';
            }
        }
        function showModal(idx, event) {
            var modalId = 'modal';
            var modalContentId = 'modal-content';
            var modalTitle = 'modal-title';
            var modal = document.getElementById(modalId);
            var modalContent = document.getElementById(modalContentId);
            var modaltitleContent = document.getElementById(modalTitle);
            var flowData = flow_data[idx];
            // 모달 제목 추가
            var title = 'ip: ' + ip_;
            
            console.log(profileData);
            console.log(idx);
            
            // profileData[idx]를 테이블로 변환
            var data = profileData[idx];
            var tableHtml = '<table border="2"><tr>';

            // 테이블 헤더 생성
            var header = ['TPort Num', 'OIP Num', 'Oport Num', 'T Avg Pkt', 'O Avg Pkt', 'T Avg Bytes', 'O Avg Bytes', 'Avg_Dur', 'S Std Pkt', 'O Std Pkt', 'S Std Bytes', 'O Std Bytes', 'std_dur']
            for (var key in header) {
                tableHtml += '<th>' + header[key] + '</th>';
            }
            tableHtml += '</tr>';
            
            var data = profileData[idx];
            let count = 0;
            
            tableHtml += '<tr>';
            
            for (var key in data) {
                tableHtml += '<td>' + data[key] + '</td>';
            }
            tableHtml += '</tr>';
            tableHtml += '</table>';
            
            var flow_header = ['SIP', 'DIP', 'proto', 'Sport', 'Dport', 'Inpkt', 'Outpkt', 'Inbytes', 'Outbytes', 'Duration', 'Label']
            tableHtml += '<table border="2"><tr>';
            
            for (var key in flow_header) {
                tableHtml += '<th>' + flow_header[key] + '</th>';
            }
            tableHtml += '</tr>';
            tableHtml += '<tr>';
            
            for (var key in flowData) {
                tableHtml += '<td>' + flowData[key] + '</td>';
                count += 1;

                // 11개의 열마다 새로운 행을 시작
                if (count % 11 === 0) {
                    tableHtml += '</tr><tr>';
                }
            }
            
            tableHtml += '</tr>';
            tableHtml += '</table>';
            

            modalContent.innerHTML = tableHtml;
            modaltitleContent.innerHTML = '<h3>' + title + '</h3>';
            modal.style.display = 'block';

            // 모달 위치 설정
            var scrollTop = window.pageYOffset || document.documentElement.scrollTop;
            var windowWidth = window.innerWidth;
            var modalWidth = modal.offsetWidth;

            modal.style.top = (event.clientY + scrollTop) + 'px';


            modal.style.left = '0px';
        }
        function closeModal() {
            var modalId = 'modal';
            var modal = document.getElementById(modalId);
            modal.style.display = 'none';
        }
        </script>
        <style>
        .row {
            display: flex;
        }
        .column {
            flex: 50%;
            padding: 20px;
            box-sizing: border-box;
        }
        .modal {
            display: none;
            position: fixed; /* fixed로 변경 */
            z-index: 1;
            left: 25%; /* 화면의 25% 지점에 위치 */
            top: 25%; /* 화면의 상단에서 25% 지점에 위치 */
            width: 80% /* 너비를 화면의 50%로 설정 */
            height: auto; /* 높이를 자동으로 설정 */
            overflow: auto;
            background-color: rgba(0,0,0,0.4);
        }
        .button-benign {
            background-color: blue;
            color: white;
        }
        .button-attack {
            background-color: red;
            color: white;
        }
        .modal-content {
            background-color: #fefefe;
            margin: 15% auto;
            padding: 20px;
            border: 1px solid #888;
            width: 100%; /* 내용을 모달의 너비에 맞춤 */
        }
        table {
            border-collapse: collapse;
            width: 100%; /* 테이블을 모달의 너비에 맞춤 */
        }

        th, td {
            border: 1px solid black;
            text-align: left;
            padding: 8px;
        }
        td {
            background-color : white;
        }

        th {
            background-color: #f2f2f2;
        }
        
        </style>
        </head><body>'''
        for quan in self.multi_set:
            # Train과 Test의 라벨 세트를 확인
            labels = self.get_quan_label(quan)
            
            # 라벨에 따른 클래스 결정
            label_class = "button-attack" if "Attack" in labels else "button-benign"
            html_content += '<div class="row">'
            # Train Column
            html_content += '<div class="column">'
            html_content += f'<button onclick="toggleData(\'{quan}\')">{quan}</button><br>'
            html_content += f'<div id="data-{quan}" style="display:none;">'
            for idx, i in enumerate(self.get_quan_index(quan)):
                _class = "button-attack" if labels[idx].upper() != 'BENIGN' else "button-benign"
                html_content += f'<button class="{_class}" onclick="showModal({i},  event)">{i}</button>'
            html_content += '</div>'
            html_content += '</div>'

        # Train 모달
        html_content += '''
        <div id="modal" class="modal">
            <div class="modal-content">
                <span onclick="closeModal()" class="close">&times;</span>
                <h3 id="modal-title"></h3>
                <p id="modal-content"></p>
            </div>
        </div>'''

        return html_content

    def save_html(self, filename):
        html_content = self.generate_html()
        with open(filename, 'w') as file:
            file.write(html_content)
    

In [66]:
class IP_compare():
    def __init__(self, ip1, ip2):
        global min_data
        self.train_ip = IP_debug(ip1)
        self.test_ip = IP_debug(ip2)
        
        self.intersection = self.train_ip.get_multi_set().intersection(self.test_ip.get_multi_set())
        self.attack_inter = set([i for i in self.intersection])
        self.normal_inter = self.intersection - self.attack_inter
        
    def generate_html(self):
        # 미리 모든 find_profile_idx 결과를 계산
        profile_data_train = {idx: self.train_ip.find_profile_idx(idx) for idx in self.train_ip.idx_index.keys() if self.train_ip.idx_quan[idx] in self.intersection}
        profile_data_test = {idx: self.test_ip.find_profile_idx(idx) for idx in self.test_ip.idx_index.keys() if self.test_ip.idx_quan[idx] in self.intersection}
        
        flow_data_train = {idx: self.train_ip.get_flow_idx(idx) for idx in self.train_ip.idx_index.keys() if self.train_ip.idx_quan[idx] in self.intersection}
        flow_data_test = {idx: self.test_ip.get_flow_idx(idx) for idx in self.test_ip.idx_index.keys() if self.test_ip.idx_quan[idx] in self.intersection}
        
        train_ip_ = self.train_ip.ip
        test_ip_ = self.test_ip.ip
        
        html_content = '<html><head>'
        html_content += '<script>'
        html_content += 'var profileDataTrain = ' + str(profile_data_train).replace("'", '"') + ';\n'
        html_content += 'var profileDataTest = ' + str(profile_data_test).replace("'", '"') + ';\n'
        html_content += 'var flow_data_train = ' + str(flow_data_train).replace("'", '"') + ';\n'
        html_content += 'var flow_data_test = ' + str(flow_data_test).replace("'", '"') + ';\n'
        html_content += 'var Total_len_train = ' + str(len(self.train_ip.get_multi_set())).replace("'", '"') + ';\n'
        html_content += 'var Total_len_test = ' + str(len(self.test_ip.get_multi_set())).replace("'", '"') + ';\n'
        html_content += 'var train_ip = ' + json.dumps(train_ip_) + ';\n'
        html_content += 'var test_ip = ' + json.dumps(test_ip_) + ';\n'
        html_content += ''' 
        
        function toggleData(quan, type) {
            var dataDiv = document.getElementById('data-' + type + '-' + quan);
            if (dataDiv) {
                dataDiv.style.display = dataDiv.style.display === 'none' ? 'block' : 'none';
            }
        }
        function showModal(idx, type, event) {
            var modalId = type === 'train' ? 'modal-train' : 'modal-test';
            var modalContentId = type === 'train' ? 'modal-content-train' : 'modal-content-test';
            var modalTitle = type === 'train' ? 'modal-title-train' : 'modal-title-test';
            var modal = document.getElementById(modalId);
            var modalContent = document.getElementById(modalContentId);
            var modaltitleContent = document.getElementById(modalTitle);
            var profileData = type === 'train' ? profileDataTrain : profileDataTest;
            var flowData = type === 'train' ? flow_data_train[idx] : flow_data_test[idx];
            var totalLen = type === 'train' ? Total_len_train : Total_len_test;
            // 모달 제목 추가
            var title = type === 'train' ? type + ' ip : ' + train_ip : type + ' ip: ' + test_ip;
            
            console.log(title)

            // profileData[idx]를 테이블로 변환
            var data = profileData[idx];
            var tableHtml = '<table border="2"><tr>';

            // 테이블 헤더 생성
            var header = ['TPort Num', 'OIP Num', 'Oport Num', 'T Avg Pkt', 'O Avg Pkt', 'T Avg Bytes', 'O Avg Bytes', 'Avg_Dur', 'S Std Pkt', 'O Std Pkt', 'S Std Bytes', 'O Std Bytes', 'std_dur']
            for (var key in header) {
                tableHtml += '<th>' + header[key] + '</th>';
            }
            tableHtml += '</tr>';
            
            var data = profileData[idx];
            let count = 0;
            
            tableHtml += '<tr>';
            
            for (var key in data) {
                tableHtml += '<td>' + data[key] + '</td>';
            }
            tableHtml += '</tr>';
            tableHtml += '</table>';
            
            var flow_header = ['SIP', 'DIP', 'proto', 'Sport', 'Dport', 'Inpkt', 'Outpkt', 'Inbytes', 'Outbytes', 'Duration', 'Label']
            tableHtml += '<table border="2"><tr>';
            
            for (var key in flow_header) {
                tableHtml += '<th>' + flow_header[key] + '</th>';
            }
            tableHtml += '</tr>';
            tableHtml += '<tr>';
            
            for (var key in flowData) {
                tableHtml += '<td>' + flowData[key] + '</td>';
                count += 1;

                // 11개의 열마다 새로운 행을 시작
                if (count % 11 === 0) {
                    tableHtml += '</tr><tr>';
                }
            }
            
            tableHtml += '</tr>';
            tableHtml += '</table>';
            

            modalContent.innerHTML = tableHtml;
            modaltitleContent.innerHTML = '<h3>' + title + ' (' + totalLen + ')' + '</h3>';
            modal.style.display = 'block';

            // 모달 위치 설정
            var scrollTop = window.pageYOffset || document.documentElement.scrollTop;
            var windowWidth = window.innerWidth;
            var modalWidth = modal.offsetWidth;

            modal.style.top = (event.clientY + scrollTop) + 'px';

            if (type === 'test') {
                // 테스트 모달의 경우 오른쪽에 위치
                modal.style.left = (windowWidth - modalWidth) + 'px';
            } else {
                // 트레인 모달의 경우 기존 위치 설정 사용
                modal.style.left = '0px';
            }
        }
        function closeModal(type) {
            var modalId = type === 'train' ? 'modal-train' : 'modal-test';
            var modal = document.getElementById(modalId);
            modal.style.display = 'none';
        }
        </script>
        <style>
        .row {
            display: flex;
        }
        .column {
            flex: 50%;
            padding: 20px;
            box-sizing: border-box;
        }
        .column.test {
            text-align: right;  /* Test 컬럼의 내용을 오른쪽 정렬 */
            padding-right:20px;
        }
        .modal {
            display: none;
            position: fixed; /* fixed로 변경 */
            z-index: 1;
            left: 25%; /* 화면의 25% 지점에 위치 */
            top: 25%; /* 화면의 상단에서 25% 지점에 위치 */
            width: 50%; /* 너비를 화면의 50%로 설정 */
            height: 600px; /* 높이를 자동으로 설정 */
            overflow: auto;
            background-color: rgba(0,0,0,0.4);
        }
        .button-benign {
            background-color: blue;
            color: white;
        }
        .button-attack {
            background-color: red;
            color: white;
        }
        .modal-content {
            background-color: #fefefe;
            margin: 15% auto;
            padding: 20px;
            border: 1px solid #888;
            width: 100%; /* 내용을 모달의 너비에 맞춤 */
        }
        table {
            border-collapse: collapse;
            width: 100%; /* 테이블을 모달의 너비에 맞춤 */
        }

        th, td {
            border: 1px solid black;
            text-align: left;
            padding: 8px;
        }
        td {
            background-color : white;
        }

        th {
            background-color: #f2f2f2;
        }
        
        </style>
        </head><body>'''
        for quan in self.intersection:
            # Train과 Test의 라벨 세트를 확인
            train_labels = self.train_ip.get_quan_label(quan)
            test_labels = self.test_ip.get_quan_label(quan)
            
            # 라벨에 따른 클래스 결정
            test_class = "button-attack" if "Attack" in test_labels else "button-benign"
            html_content += '<div class="row">'
            # Train Column
            html_content += '<div class="column">'
            html_content += f'<button onclick="toggleData(\'{quan}\', \'train\')">{quan} (Train)</button><br>'
            html_content += f'<div id="data-train-{quan}" style="display:none;">'
            for idx, i in enumerate(self.train_ip.get_quan_index(quan)):
                train_class = "button-attack" if train_labels[idx].upper() != 'BENIGN' else "button-benign"
                html_content += f'<button class="{train_class}" onclick="showModal({i}, \'train\', event)">{i}</button>'
            html_content += '</div>'
            html_content += '</div>'

            # Test Column
            html_content += '<div class="column test">'
            html_content += f'<button onclick="toggleData(\'{quan}\', \'test\')">{quan} (Test)</button><br>'
            html_content += f'<div id="data-test-{quan}" style="display:none;">'
            for idx, i in enumerate(self.test_ip.get_quan_index(quan)):
                test_class = "button-attack" if test_labels[idx].upper() != 'BENIGN' else "button-benign"
                html_content += f'<button class="{test_class}" onclick="showModal({i}, \'test\', event)">{i}</button>'
            html_content += '</div>'
            html_content += '</div>'
            html_content += '</div>'  # End of Row

        # Train 모달
        html_content += '''
        <div id="modal-train" class="modal">
            <div class="modal-content">
                <span onclick="closeModal('train')" class="close">&times;</span>
                <h3 id="modal-title-train"></h3>
                <p id="modal-content-train"></p>
            </div>
        </div>'''

        # Test 모달
        html_content += '''
        <div id="modal-test" class="modal">
            <div class="modal-content">
                <span onclick="closeModal('test')" class="close">&times;</span>
                <h3 id="modal-title-test"></h3>
                <p id="modal-content-test"></p>
            </div>
        </div>
        </body></html>'''
        return html_content

    def save_html(self, filename):
        html_content = self.generate_html()
        with open(filename, 'w') as file:
            file.write(html_content)

# bot1 = 'ATTACK_147.32.84.165_train_bot5'
# bot2 = 'ATTACK_147.32.84.165_test_bot13'

# b = IP_compare(f'{bot1}', f'{bot2}')
# b.save_html(f'./Debug/{data_path}/False-Positive/{count_prot}-{bot1}-{bot2}.html')

In [70]:
def all_do(dataset_path, min_data, attack, change_feature, add_src, separate_attackIP, count_prot, train_window, test_window, logN, real_time, using_minmax, using_quan, p0):
    parameter = f"cf({change_feature})_sepIP({separate_attackIP})_min({min_data})_mm({using_minmax})"
    
    if using_quan == 'log':
        parameter += f'_pro({count_prot})_as({add_src})_log({logN})'
    elif using_quan == 'bayesian':
        parameter += f'_pro({count_prot})_as({add_src})_p0({p0})'
    
    global train_dataset
    global test_dataset
    global train_multi
    global test_multi
    global train_data
    global test_data
    global train_raw
    global test_raw
    global train_key
    global test_key
    
    with open(f"./debug_data/{dataset_path}/{parameter}/train_multi_dict_attack{attack}.pkl", 'rb') as f:
        train_multi = pickle.load(f)
    with open(f"./debug_data/{dataset_path}/{parameter}/test_multi_dict_attack{attack}.pkl", 'rb') as f:
        test_multi = pickle.load(f)
        
    with open(f"./debug_data/{dataset_path}/{parameter}/train_data_attack{attack}.pkl", 'rb') as f:
        train_data = pickle.load(f)
    with open(f"./debug_data/{dataset_path}/{parameter}/test_data_attack{attack}.pkl", 'rb') as f:
        test_data = pickle.load(f)
        
    parameter = f"cf({change_feature})_sepIP({separate_attackIP})_min({min_data})_mm({using_minmax})"

    train_raw = []
    train_key = []

    test_raw = []
    test_key = []

    #데이터 불러오기
    folder = f'./preprocessing/{dataset_path}/profiling/{parameter}'

    # 'train_feature'으로 시작하는 모든 파일 찾기
    train_ffiles = glob.glob(os.path.join(folder, 'train_feature*'))
    train_ffiles.sort()
    for file in train_ffiles:
        with open(file, 'rb') as f:
            train_raw += pickle.load(f)

    # 'train_key'로 시작하는 모든 파일 찾기
    train_kfiles = glob.glob(os.path.join(folder, 'train_key*'))
    train_kfiles.sort()
    for file in train_kfiles:
        with open(file, 'rb') as f:
            train_key += pickle.load(f)

    # 'test_feature'로 시작하는 모든 파일 찾기
    test_ffiles = glob.glob(os.path.join(folder, 'test_feature*'))
    test_ffiles.sort()
    for file in test_ffiles:
        with open(file, 'rb') as f:
            test_raw += pickle.load(f)

    # 'test_key'로 시작하는 모든 파일 찾기
    test_kfiles = glob.glob(os.path.join(folder, 'test_key*'))
    test_kfiles.sort()
    for file in test_kfiles:
        with open(file, 'rb') as f:
            test_key += pickle.load(f)
            
    train_dataset = dict()
    test_dataset = dict()

    train_list = os.listdir(f'./dataset/{dataset_path}/train/')
    test_list = os.listdir(f'./dataset/{dataset_path}/test/')

    for file in train_list:
        file_name = file.split('.')[0]
        data = pd.read_csv(f"./dataset/{dataset_path}/train/{file}")
        train_dataset[file_name] = data
    
    for file in test_list:
        file_name = file.split('.')[0]
        data = pd.read_csv(f"./dataset/{dataset_path}/test/{file}")
        test_dataset[file_name] = data
    
    if using_quan == 'log':
        file_name = f"log({logN})-cf({change_feature})-prot({count_prot})-sepIP({separate_attackIP})-min({min_data})-atk({attack})-window({train_window}-{test_window})-mm({using_minmax}).csv"
    elif using_quan == 'bayesian':
        file_name = f"p0({p0})_as({add_src})-cf({change_feature})-prot({count_prot})-sepIP({separate_attackIP})-min({min_data})-atk({attack})-window({train_window}-{test_window})-mm({using_minmax}).csv"

    save_file = f"./result/{dataset_path}/{using_quan}_{file_name}.csv"
    data = pd.read_csv(fr'{save_file}', encoding='utf-8')
    
    if using_quan == 'log':
        parameter = f"log({logN})-cf({change_feature})-prot({count_prot})-sepIP({separate_attackIP})-min({min_data})-atk({attack})-window({train_window}-{test_window})-mm({using_minmax})"
    elif using_quan == 'bayesian':
        parameter = f"p0({p0})_as({add_src})-cf({change_feature})-prot({count_prot})-sepIP({separate_attackIP})-min({min_data})-atk({attack})-window({train_window}-{test_window})-mm({using_minmax})"
    
    # 여기서 고르는 방식을 바꿔야함.
    fp_data = data[(data['Test IP Label'] == 'BENIGN') & (data['Max Sim'] != '-')]
    print(len(fp_data))
    
    if not os.path.isdir(f'./Debug/{dataset_path}/'):
        os.mkdir(f'./Debug/{dataset_path}/')
    
    if not os.path.isdir(f'./Debug/{dataset_path}/{parameter}'):
        os.mkdir(f'./Debug/{dataset_path}/{parameter}')
        os.mkdir(f'./Debug/{dataset_path}/{parameter}/False-Positive')
        os.mkdir(f'./Debug/{dataset_path}/{parameter}/True-Positive')

    for index, row in fp_data.iterrows():
        # 'MAX' 컬럼에서 set을 추출하고, 'Test' 컬럼의 값을 가져옴
        max_set = eval(row['Max IP'])  # set 형식의 문자열을 실제 set 객체로 변환
        test_IP = row['Test IP']
        sim = row['Max Sim']
        
        for train_ip in max_set:
            b = IP_compare(f'{train_ip}', f'{test_IP}')
            b.save_html(f'./Debug/{dataset_path}/{parameter}/False-Positive/{sim}-{train_ip}-{test_IP}.html')
        
    # 여기서 고르는 방식을 바꿔야함.
    tp_data = data[(data['Test IP Label'] != 'BENIGN') & (data['Max Sim'] != '-')]

    for index, row in tp_data.iterrows():
        # 'MAX' 컬럼에서 set을 추출하고, 'Test' 컬럼의 값을 가져옴
        max_set = eval(row['Max IP'])  # set 형식의 문자열을 실제 set 객체로 변환
        test_IP = row['Test IP']
        sim = row['Max Sim']
        
        for train_ip in max_set:
            b = IP_compare(f'{train_ip}', f'{test_IP}')
            b.save_html(f'./Debug/{dataset_path}/{parameter}/True-Positive/{sim}-{train_ip}-{test_IP}.html')

In [68]:
global min_data
min_data = 7
change_feature = False
seperate_attackIP = True
count_prot = True
using_minmax = True
add_src = True
real_time = 0
attack = 1 # 0이 정상 1이 공격 2가 혼합
train_window = 0
test_window = 10
p0 = 0.05
logN = 1.2

In [69]:
for data in ['MTA']:
    print(f"{data} 시작!")
    for using_quan in ['bayesian']:
        all_do(data, min_data, attack, change_feature, add_src, seperate_attackIP, count_prot, train_window, test_window, logN, real_time, using_minmax, using_quan, p0)
    print(f"{data} 끝!")

MTA 시작!
0
10.6.21.101
10.6.26.101
0 490
4 490
0 261
4 261
10.6.19.101
10.6.26.101
0 254
1 254
0 261
1 261
10.6.19.101
10.7.1.101
1 254
1 316


C:\Users\seclab\AppData\Local\Temp\ipykernel_23952\3102080725.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.flow_data.fillna(-1, inplace=True)


10.6.20.101
10.7.4.101
1 334
3 334
1 303
3 303
162.0.237.236
162.0.237.236
9 109567
10 109567
11 109567
12 109567
13 109567
26 109567
27 109567
28 109567
29 109567
30 109567
50 109567
51 109567
52 109567
71 109567
72 109567
73 109567
74 109567
75 109567
76 109567
77 109567
80 109567
89 109567
90 109567
91 109567
100 109567
101 109567
102 109567
103 109567
104 109567
105 109567
109 109567
111 109567
112 109567
130 109567
131 109567
132 109567
133 109567
134 109567
135 109567
136 109567
137 109567
138 109567
139 109567
140 109567
146 109567
157 109567
158 109567
159 109567
160 109567
161 109567
162 109567
173 109567
174 109567
181 109567
182 109567
183 109567
184 109567
185 109567
186 109567
187 109567
188 109567
189 109567
190 109567
191 109567
192 109567
200 109567
201 109567
202 109567
203 109567
204 109567
209 109567
210 109567
218 109567
227 109567
228 109567
229 109567
230 109567
231 109567
232 109567
233 109567
234 109567
235 109567
236 109567
237 109567
238 109567
239 109567
240 

3121 109567
3123 109567
3125 109567
3150 109567
3151 109567
3161 109567
3162 109567
3163 109567
3164 109567
3165 109567
3166 109567
3167 109567
3168 109567
3181 109567
3182 109567
3183 109567
3184 109567
3231 109567
3232 109567
3233 109567
3234 109567
3235 109567
3257 109567
3275 109567
3295 109567
3296 109567
3297 109567
3298 109567
3308 109567
3321 109567
3322 109567
3325 109567
3326 109567
3327 109567
3335 109567
3336 109567
3337 109567
3350 109567
3351 109567
3352 109567
3353 109567
3354 109567
3355 109567
3356 109567
3357 109567
3358 109567
3361 109567
3362 109567
3404 109567
3405 109567
3406 109567
3414 109567
3415 109567
3416 109567
3419 109567
3432 109567
3433 109567
3441 109567
3442 109567
3443 109567
3444 109567
3445 109567
3446 109567
3447 109567
3448 109567
3466 109567
3469 109567
3470 109567
3471 109567
3486 109567
3487 109567
3491 109567
3492 109567
3493 109567
3515 109567
3529 109567
3530 109567
3531 109567
3532 109567
3533 109567
3534 109567
3553 109567
3556 109567
3557

6800 109567
6801 109567
6802 109567
6803 109567
6804 109567
6805 109567
6806 109567
6807 109567
6808 109567
6817 109567
6818 109567
6819 109567
6820 109567
6821 109567
6822 109567
6823 109567
6824 109567
6847 109567
6848 109567
6849 109567
6850 109567
6851 109567
6852 109567
6853 109567
6859 109567
6860 109567
6861 109567
6862 109567
6863 109567
6864 109567
6865 109567
6866 109567
6867 109567
6875 109567
6876 109567
6877 109567
6885 109567
6930 109567
6931 109567
6933 109567
6934 109567
6935 109567
6936 109567
6937 109567
6938 109567
6939 109567
6940 109567
6941 109567
6942 109567
6955 109567
6965 109567
6966 109567
6967 109567
6968 109567
6976 109567
6977 109567
6978 109567
6979 109567
6980 109567
6981 109567
6982 109567
6983 109567
6984 109567
6985 109567
6986 109567
6995 109567
6996 109567
6997 109567
7002 109567
7010 109567
7011 109567
7023 109567
7024 109567
7025 109567
7026 109567
7027 109567
7028 109567
7029 109567
7030 109567
7031 109567
7032 109567
7033 109567
7034 109567
7035

...

113583 121938
113584 121938
113585 121938
113586 121938
113587 121938
113588 121938
113589 121938
113601 121938
113602 121938
113603 121938
113604 121938
113606 121938
113607 121938
113628 121938
113630 121938
113631 121938
113636 121938
113637 121938
113639 121938
113640 121938
113641 121938
113642 121938
113643 121938
113645 121938
113647 121938
113648 121938
113650 121938
113651 121938
113655 121938
113656 121938
113657 121938
113671 121938
113672 121938
113674 121938
113675 121938
113676 121938
113677 121938
113678 121938
113679 121938
113680 121938
113681 121938
113682 121938
113683 121938
113684 121938
113685 121938
113686 121938
113687 121938
113688 121938
113689 121938
113690 121938
113691 121938
113693 121938
113694 121938
113695 121938
113696 121938
113697 121938
113698 121938
113699 121938
113700 121938
113701 121938
113702 121938
113704 121938
113705 121938
113707 121938
113708 121938
113709 121938
113710 121938
113711 121938
113712 121938
113713 121938
113716 121938
113718

116744 121938
116745 121938
116746 121938
116747 121938
116748 121938
116750 121938
116751 121938
116752 121938
116753 121938
116754 121938
116755 121938
116756 121938
116757 121938
116758 121938
116759 121938
116762 121938
116763 121938
116764 121938
116765 121938
116766 121938
116767 121938
116768 121938
116769 121938
116770 121938
116771 121938
116772 121938
116773 121938
116774 121938
116775 121938
116776 121938
116779 121938
116780 121938
116781 121938
116782 121938
116783 121938
116784 121938
116785 121938
116787 121938
116788 121938
116797 121938
116798 121938
116799 121938
116800 121938
116801 121938
116802 121938
116805 121938
116817 121938
116818 121938
116820 121938
116821 121938
116822 121938
116823 121938
116824 121938
116825 121938
116826 121938
116827 121938
116828 121938
116829 121938
116830 121938
116831 121938
116832 121938
116839 121938
116840 121938
116841 121938
116842 121938
116847 121938
116848 121938
116849 121938
116850 121938
116851 121938
116852 121938
116865

118197 121938
118198 121938
118199 121938
118209 121938
118210 121938
118211 121938
118212 121938
118213 121938
118214 121938
118215 121938
118216 121938
118217 121938
118218 121938
118219 121938
118220 121938
118221 121938
118223 121938
118224 121938
118225 121938
118226 121938
118227 121938
118228 121938
118229 121938
118230 121938
118231 121938
118232 121938
118235 121938
118236 121938
118237 121938
118238 121938
118239 121938
118240 121938
118241 121938
118242 121938
118243 121938
118244 121938
118245 121938
118246 121938
118247 121938
118248 121938
118249 121938
118250 121938
118251 121938
118252 121938
118253 121938
118254 121938
118255 121938
118256 121938
118257 121938
118258 121938
118259 121938
118260 121938
118261 121938
118269 121938
118270 121938
118271 121938
118272 121938
118273 121938
118274 121938
118275 121938
118276 121938
118280 121938
118281 121938
118282 121938
118285 121938
118286 121938
118287 121938
118288 121938
118289 121938
118290 121938
118291 121938
118292

120575 121938
120576 121938
120577 121938
120578 121938
120579 121938
120580 121938
120590 121938
120591 121938
120592 121938
120593 121938
120594 121938
120596 121938
120613 121938
120614 121938
120615 121938
120618 121938
120619 121938
120620 121938
120621 121938
120622 121938
120623 121938
120624 121938
120625 121938
120626 121938
120627 121938
120628 121938
120640 121938
120641 121938
120642 121938
120643 121938
120655 121938
120658 121938
120659 121938
120660 121938
120661 121938
120662 121938
120663 121938
120671 121938
120672 121938
120673 121938
120674 121938
120675 121938
120676 121938
120677 121938
120678 121938
120679 121938
120691 121938
120702 121938
120703 121938
120704 121938
120705 121938
120706 121938
120707 121938
120708 121938
120709 121938
120710 121938
120713 121938
120714 121938
120715 121938
120716 121938
120724 121938
120725 121938
120726 121938
120727 121938
120728 121938
120729 121938
120737 121938
120738 121938
120739 121938
120740 121938
120741 121938
120742

MTA 끝!


In [45]:
dataset_path = 'All-CTU'
parameter = 'p0(0.05)_as(True)-cf(False)-prot(True)-sepIP(True)-min(5)-atk(1)-window(0-10)-zero(False)-mm(True)-vic(False)'
sim = 'test'
b = IP_compare(f'ATTACK_147.32.84.165_train_bot4', f'147.32.86.5_test_bot8')
b.save_html(f'./Debug/{dataset_path}/{parameter}/True-Positive/{sim}-ATTACK_147.32.84.165_train_bot4-147.32.86.5_test_bot8.html')

147.32.84.165


C:\Users\seclab\AppData\Local\Temp\ipykernel_23952\3102080725.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.flow_data.fillna(-1, inplace=True)


147.32.86.5
2679 3350
2680 3350
2681 3350
8 1500
9 1500
10 1500
11 1500
12 1500
26 1500
27 1500
28 1500
29 1500
98 1500
99 1500
152 1500
172 1500
185 1500
186 1500
187 1500
188 1500
201 1500
202 1500
203 1500
204 1500
212 1500
245 1500
459 1500
460 1500
461 1500
462 1500
463 1500
464 1500
465 1500
495 1500
1237 1500
1238 1500
1261 1500
1262 1500
1263 1500
1264 1500
1265 1500
1283 1500
1385 1500
1386 1500
1387 1500
1388 1500
1389 1500
1409 1500
1410 1500
1411 1500
1412 1500
1413 1500
1453 1500
1454 1500
1494 1500
1495 1500


In [46]:
with open('./preprocessing/All-CTU/Bayesian/p0(0.05)_atk(1)_mm(True)_bayes.pkl', 'rb') as f:
    model = pickle.load(f)

In [61]:
column_dict = {0 : 'card_target_port', 1:'card_opposite_ip',
2:'card_opposite_port',
3:'avg_target_pkts',
4:'avg_opposite_pkts',
5:'avg_target_bytes',
6:'avg_opposite_bytes',
7:'avg_dur',
8:'std_target_pkts',
9:'std_opposite_pkts',
10:'std_target_bytes',
11:'std_opposite_bytes',
12:'std_dur',
13:'max_target_pkts',
14:'max_opposite_pkts',
15:'max_target_bytes',
16:'max_opposite_bytes',
17:'max_duration',
18:'min_target_pkts',
19:'min_opposite_pkts',
20:'min_target_bytes',
21:'min_opposite_bytes',
22:'min_duration'}

In [63]:
import numpy as np
for idx, i in enumerate(model.boundary):
    print(column_dict[idx])
    print(np.diff(model.boundary[i]))

card_target_port
[]
card_opposite_ip
[]
card_opposite_port
[]
avg_target_pkts
[1.0000e-01 2.0000e-01 2.0000e-01 2.0000e-01 4.0000e-01 2.0000e-01
 6.0000e-01 2.0000e-01 2.0000e-01 2.0000e-01 2.0000e-01 2.0000e-01
 4.0000e-01 2.0000e-01 2.0000e-01 2.0000e-01 4.0000e-01 4.0000e-01
 3.8000e+00 1.2000e+00 4.0000e-01 6.0000e-01 6.0000e-01 1.6000e+00
 1.2000e+00 1.6000e+00 1.0000e+00 8.0000e-01 1.8000e+00 6.4000e+00
 6.0000e+00 7.4000e+00 6.0000e-01 4.0000e+00 1.3200e+01 1.3900e+01
 9.2000e+00 7.7000e+00 1.4800e+01 2.1000e+00 9.3600e+01 6.2300e+01
 3.7000e+00 1.6030e+02 1.3700e+01 5.5810e+02 1.5200e+01 1.2818e+03
 1.7000e+00]
avg_opposite_pkts
[3.00000e-01 2.00000e-01 2.00000e-01 2.00000e-01 2.00000e-01 2.00000e-01
 2.00000e-01 2.00000e-01 2.00000e-01 2.00000e-01 2.00000e-01 2.00000e-01
 2.00000e-01 2.00000e-01 2.00000e-01 2.00000e-01 4.00000e-01 4.00000e-01
 8.00000e-01 4.00000e-01 4.00000e-01 6.00000e-01 1.60000e+00 2.00000e-01
 4.00000e-01 4.00000e-01 2.00000e-01 4.00000e-01 6.00000e-01 1.